# R in Jupyter Notebook

This notebook shows:
- How to install R package in Python  
- Import and call `R` functions in Python.  
- Transfer data from Python to `R`, run `R` functions, and transfer the results back to Python.

In [1]:
# uncomment and run the following line to install ry2 if you haven't installed it yet

# !pip install rpy2

In [2]:
import numpy as np
from rpy2.robjects.packages import importr

## Install R package

In [3]:
utils = importr('utils')
utils.install_packages('stats')

--- Please select a CRAN mirror for use in this session ---


<rpy2.rinterface_lib.sexp.NULLType object at 0x108724240> [RTYPES.NILSXP]

## Import and call `R` functions in Python

In this section, we compare `R` `p.adjust` function with its corresponding python implementation to validate two implementations are identical.

In [4]:
# https://stackoverflow.com/a/21739593
def correct_pvalues_for_multiple_testing(pvalues, correction_type = "Benjamini-Hochberg"):                
    """                                                                                                   
    consistent with R - print correct_pvalues_for_multiple_testing([0.0, 0.01, 0.029, 0.03, 0.031, 0.05, 0.069, 0.07, 0.071, 0.09, 0.1]) 
    """
    from numpy import array, empty
    pvalues = array(pvalues) 
    n = pvalues.shape[0]                                                           
    new_pvalues = np.zeros(n)
    if correction_type == "Bonferroni":                                                                   
        new_pvalues = n * pvalues
    elif correction_type == "Bonferroni-Holm":                                                            
        values = [ (pvalue, i) for i, pvalue in enumerate(pvalues) ]                                      
        values.sort()
        for rank, vals in enumerate(values):                                                              
            pvalue, i = vals
            new_pvalues[i] = (n-rank) * pvalue                                                            
    elif correction_type == "Benjamini-Hochberg":                                                         
        values = [ (pvalue, i) for i, pvalue in enumerate(pvalues) ]                                      
        values.sort()
        values.reverse()                                                                                  
        new_values = []
        for i, vals in enumerate(values):                                                                 
            rank = n - i
            pvalue, index = vals                                                                          
            new_values.append((n/rank) * pvalue)                                                          
        for i in range(0, int(n)-1):  
            if new_values[i] < new_values[i+1]:                                                           
                new_values[i+1] = new_values[i]                                                           
        for i, vals in enumerate(values):
            pvalue, index = vals
            new_pvalues[index] = new_values[i]                                                                                                                  
    return new_pvalues

In [5]:
stats = importr('stats')

during import `R` function name may change, search for the right name for `p.adjust`

In [6]:
methods = list(dir(stats))
[s for s in methods if "adjust" in s]

['p_adjust', 'p_adjust_methods']

In [7]:
pvalues=np.random.random(100)

In [8]:
from rpy2.robjects.vectors import FloatVector

p_adjust_R = stats.p_adjust(FloatVector(pvalues), method = 'BH')
p_adjust_P = correct_pvalues_for_multiple_testing(pvalues)

In [9]:
p_adjust_R-p_adjust_P

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [10]:
np.sum(np.abs(p_adjust_R-p_adjust_P))

0.0

this shows that the two implementations are identical

## Transfer data from Python to `R` and transfer the results back to Python

In [11]:
%load_ext rpy2.ipython

run `R` magic. `pvalues` is input and `p_adjust_R1` is output which are seen in Python. You can run `R` code in the cell below.

In [12]:
%%R -i pvalues -o p_adjust_R1
p_adjust_R1 <- p.adjust(pvalues, method='BH')

In [13]:
p_adjust_R1-p_adjust_R

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [14]:
np.sum(np.abs(p_adjust_R-p_adjust_R1))

0.0

this shows that calling method in R and importing R method to Python return identical results.